[![Open in Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/pantat88/segsmaker/blob/main/Segsmaker_ComfyUI.ipynb)

## Conda

In [ ]:
from IPython.display import HTML, display
import subprocess
def ass(command, message, color):
    display(HTML(f"<span style='color:{color};'>・・・ {message} ・・・</span>"))
    result = subprocess.run(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print("done")
ass('conda install -q -y conda=23.11.0 glib gxx_linux-64 pyngrok ffmpeg av gst-libav imageio imageio-ffmpeg psutil', 'Installing Conda', 'cyan')
ass('conda install -q -y -n base python=3.10.12', 'Installing Python 3.10.12', '#D48900')
ass('conda clean -y --all', 'Cleaning Conda Environment', '#66ff00')
ass('curl -Lo ~/.ipython/profile_default/startup/pantat88.py https://github.com/gutris1/segsmaker/raw/main/ui/pantat88.py', 'Gathering Magic', 'red')
get_ipython().kernel.do_shutdown(True)

## Clone ComfyUI

In [ ]:
%say ・・・ {red} Cloning ComfyUI {d} ・・・ {red}
%cd -q /home/studio-lab-user
!git clone -q https://github.com/comfyanonymous/ComfyUI

%say ・・・ {red} Installing Requirements {d} ・・・ {red}
%cd -q ~/ComfyUI
%download https://github.com/gutris1/segsmaker/raw/main/ui/cui.zip
!unzip -qo cui.zip && rm cui.zip
!pip install https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118 > /dev/null

%say ・・・ {red} Symlink to Temporary Folder {d} ・・・ {red}
!rm -rf ~/tmp/* ~/tmp ~/ComfyUI/models/checkpoints ~/ComfyUI/models/loras ~/ComfyUI/models/controlnet
!ln -vs /tmp ~/tmp > /dev/null ; ln -vs /tmp/Lora ~/ComfyUI/models/loras > /dev/null
!ln -vs /tmp/models ~/ComfyUI/models/checkpoints > /dev/null ; ln -vs /tmp/ControlNet ~/ComfyUI/models/controlnet > /dev/null
''' Embeddings ''' 
%say ・・・ {red} Downloading Embeddings {d} ・・・ {red}
%cd -q ~/ComfyUI/models/embeddings
%download https://huggingface.co/pantat88/ui/resolve/main/embeddings.zip
!unzip -qo embeddings.zip && rm embeddings.zip
%download https://civitai.com/api/download/models/150491 edgQuality.pt
''' Upscaler '''
%say ・・・ {red} Downloading Upscalers {d} ・・・ {red}
%cd -q ~/ComfyUI/models/upscale_models
%download ~/ComfyUI/asd/upscaler.txt
''' VAE '''
%say ・・・ {red} Downloading VAE {d} ・・・ {red}
%cd -q ~/ComfyUI/models/vae
%download https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors
''' Custom Nodes '''
%say ・・・ {red} Cloning ComfyUI Custom Nodes {d} ・・・ {red}
%cd -q ~/ComfyUI/custom_nodes
%clone ~/ComfyUI/asd/custom_nodes.txt
%download ~/ComfyUI/asd/custom_nodes_models.txt

%say ・・・ {red} Installing Custom Nodes Dependencies {d} ・・・ {red}
%cd -q ~/ComfyUI
!pip install opencv-python==4.7.0.72 transformers==4.26.1 tokenizers clip-interrogator > /dev/null
![ -f "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py" ] && python "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py" > /dev/null

!rm -rf ~/.cache/*
%say ・・・ {red} Done {d} ・・・ {red}

## *ControlNET models*
###### temporary

In [ ]:
%tempe
%cd -q ~/ComfyUI/models/controlnet
%download ~/ComfyUI/asd/controlnet.txt

## *Checkpoint*
###### temporary

In [ ]:
%tempe
%cd -q ~/ComfyUI/models/checkpoints

%download https://civitai.com/api/download/models/147913 meichidarkMixFp16_45.safetensors

## *Lora*
###### temporary

In [ ]:
%tempe
%cd -q ~/ComfyUI/models/loras

%download https://civitai.com/api/download/models/87153  add_detail.safetensors
%download https://civitai.com/api/download/models/171989 detail_slider_v4.safetensors
%download https://civitai.com/api/download/models/62833  Detail_Tweaker.safetensors
%download https://civitai.com/api/download/models/86247  flat2.safetensors

## Run ComfyUI

In [ ]:
ngrok_token = "PASTE_YOUR_NGROK_TOKEN_HERE"

from threading import Timer
from queue import Queue
from pyngrok import ngrok

def ngrok_tunnel(port,queue,auth_token):
    ngrok.set_auth_token(auth_token)
    url = ngrok.connect(port)
    queue.put(url)

ngrok_output_queue = Queue()
ngrok_thread = Timer(2, ngrok_tunnel, args=(8188, ngrok_output_queue, ngrok_token))
ngrok_thread.start()
ngrok_thread.join()
print(ngrok_output_queue.get())

%cd -q ~/ComfyUI
!python main.py --dont-print-server

## Extras

#### ```zip``` Output images

In [ ]:
%%zipping

input_folder  = '/home/studio-lab-user/ComfyUI/output'
output_folder = '/home/studio-lab-user'

#### Checking and Cleaning Storage

In [ ]:
''' Check Storage '''

!rm -rf ~/.cache/*
%storage /home/studio-lab-user

In [ ]:
''' Delete output images '''

!rm -rf ~/ComfyUI/output/* ~/.cache/*

In [ ]:
''' Delete ComfyUI '''

!rm -rf ~/tmp/* ~/ComfyUI ~/tmp ~/.cache/*
%cd -q ~

In [ ]:
''' DELETE EVERYTHING , restart jupyterlab afterward '''

!rm -rf ~/tmp/* ~/ComfyUI ~/tmp
!rm -rf ~/.cache/* ~/.config/* ~/.conda/*
!find ~/ -type d -name .ipynb_checkpoints -exec rm -rf {} \; > /dev/null 2>&1
!find ~/* -type f ! -name '*.ipynb' -exec rm -f {} + > /dev/null 2>&1